In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Billioners_data").getOrCreate()
spark

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.functions import col, when
from pyspark.sql.types import *

In [3]:
#reading the file 
data=spark.read.csv(r"C:\Users\DELL\Downloads\cvs files\Billionaires Statistics Dataset.csv",header=True,inferSchema=True)
data.show()

+----+----------+--------------------+--------------------+---+-------------+-----------+--------------------+--------------------+--------------------+--------------------+--------+------+------+---------------+------------------+---------+--------------------+-------------+----------+--------------------+---------+----------+--------+-----------+------------------+--------------------+-----------------------------------+------------------------------------------+-----------------------+---------------------------+----------------------+------------------+----------------+-----------------+
|rank|finalWorth|            category|          personName|age|      country|       city|              source|          industries|countryOfCitizenship|        organization|selfMade|status|gender|      birthDate|          lastName|firstName|               title|         date|     state|residenceStateRegion|birthYear|birthMonth|birthDay|cpi_country|cpi_change_country|         gdp_country|gross_terti

In [11]:
data.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- finalWorth: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- personName: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- source: string (nullable = true)
 |-- industries: string (nullable = true)
 |-- countryOfCitizenship: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- selfMade: boolean (nullable = true)
 |-- status: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthDate: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- residenceStateRegion: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- birthMonth: integer (nullable = true)
 |-- birthDay: integer (nullable = true)
 |-- cpi_country: d

In [12]:
data.describe().show()

+-------+-----------------+-----------------+----------+------------------+------------------+-------+--------+-----------------+----------+--------------------+------------+------+------+-------------+-------------+---------+-------------+-------------+-------+--------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+-----------------------------------+------------------------------------------+-----------------------+---------------------------+----------------------+-------------------+------------------+------------------+
|summary|             rank|       finalWorth|  category|        personName|               age|country|    city|           source|industries|countryOfCitizenship|organization|status|gender|    birthDate|     lastName|firstName|        title|         date|  state|residenceStateRegion|         birthYear|        birthMonth|         birthDay|       cpi_country|cpi_change_country|        gdp_c

In [13]:
#To calculate the null values in the column
null_counts=[sum(col(i).isNull().cast("int")).alias(i) for i in data.columns]
result = data.select(null_counts)
result.show()

+----+----------+--------+----------+---+-------+----+------+----------+--------------------+------------+--------+------+------+---------+--------+---------+-----+----+-----+--------------------+---------+----------+--------+-----------+------------------+-----------+-----------------------------------+------------------------------------------+-----------------------+---------------------------+----------------------+------------------+----------------+-----------------+
|rank|finalWorth|category|personName|age|country|city|source|industries|countryOfCitizenship|organization|selfMade|status|gender|birthDate|lastName|firstName|title|date|state|residenceStateRegion|birthYear|birthMonth|birthDay|cpi_country|cpi_change_country|gdp_country|gross_tertiary_education_enrollment|gross_primary_education_enrollment_country|life_expectancy_country|tax_revenue_country_country|total_tax_rate_country|population_country|latitude_country|longitude_country|
+----+----------+--------+----------+---+---

In [14]:
# All the operations are wrtiien in a function
#to get the distint values in the column
def distint_counts(data_frame:DataFrame,column:str):
    try:
        distinct_values=data_frame.select(column).distinct()
        distinct_count=distinct_values.count()
        null_values=data_frame.filter(data_frame[column].isNull()).count() #Null count
        dtpyes_column=data_frame.schema[column].dataType
        count_column=data_frame.select("age").count() #To get the total count of the data
        result = {
            "Column Name": column,
            "Distinct Count": distinct_count,
            "Null Values Count": null_values,
            "Data Type of Column":dtpyes_column,
            "Total Values in the Column": count_column
        }
        return result
    except Exception as e:  
        return {    
            "Error the column name should be given in str  as will data should be in data frame": str(e)
        }
distint_counts(data,"age")

{'Column Name': 'age',
 'Distinct Count': 80,
 'Null Values Count': 65,
 'Data Type of Column': IntegerType(),
 'Total Values in the Column': 2640}

In [15]:
#using filter to see the null values 
data.filter(data["age"].isNull()).show() 

+----+----------+--------------------+--------------------+----+--------------------+------------+--------------------+--------------------+--------------------+------------+--------+--------------------+------+---------+----------------+-----------------+-------+-------------+-----+--------------------+---------+----------+--------+-----------+------------------+--------------------+-----------------------------------+------------------------------------------+-----------------------+---------------------------+----------------------+------------------+----------------+-----------------+
|rank|finalWorth|            category|          personName| age|             country|        city|              source|          industries|countryOfCitizenship|organization|selfMade|              status|gender|birthDate|        lastName|        firstName|  title|         date|state|residenceStateRegion|birthYear|birthMonth|birthDay|cpi_country|cpi_change_country|         gdp_country|gross_tertiary_ed

In [38]:
#dropping the columns which are meaningless
data=data.drop("lastName","firstName","date")

In [17]:
#Status: "D" represents self-made billionaires (Founders/Entrepreneurs) & "U" indicates inherited or unearned wealth.tatus: "D" represents self-made billionaires (Founders/Entrepreneurs) and "U" indicates inherited or unearned wealth.
data=data.withColumn("status_billioners",when(col("status").contains("U"),"unearned_wealth").when(col("status").contains("D"),"self-made").otherwise(col("status")))

In [18]:
#As the type of birthdate is in string we need to change it to date formate
from pyspark.sql.functions import to_date
data.select("birthDate").dtypes
data=data.withColumn("birthDate", regexp_replace("birthDate", "0:00", ""))
#trimming the extra spaces
data = data.withColumn("birthDate", trim(col("birthDate")))
#converting the string formate to date type
data=data.withColumn("birthDate", to_date("birthDate", "M/d/yyyy"))

In [19]:
data.show()

+----+----------+--------------------+--------------------+---+-------------+-----------+--------------------+--------------------+--------------------+--------------------+--------+------+------+----------+--------------------+-------------+----------+--------------------+---------+----------+--------+-----------+------------------+--------------------+-----------------------------------+------------------------------------------+-----------------------+---------------------------+----------------------+------------------+----------------+-----------------+-----------------+
|rank|finalWorth|            category|          personName|age|      country|       city|              source|          industries|countryOfCitizenship|        organization|selfMade|status|gender| birthDate|               title|         date|     state|residenceStateRegion|birthYear|birthMonth|birthDay|cpi_country|cpi_change_country|         gdp_country|gross_tertiary_education_enrollment|gross_primary_education_e

In [21]:
#group by
data.groupBy("category").count().show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|             Telecom|   31|
|     Metals & Mining|   74|
|     Food & Beverage|  212|
|              Energy|  100|
|          Healthcare|  201|
|Media & Entertain...|   91|
|              Sports|   39|
|         Real Estate|  193|
|         Diversified|  187|
|          Automotive|   73|
|  Gambling & Casinos|   25|
|Construction & En...|   45|
|          Technology|  314|
|    Fashion & Retail|  266|
|             Service|   53|
|Finance & Investm...|  372|
|       Manufacturing|  324|
|           Logistics|   40|
+--------------------+-----+



In [31]:
data.filter(data["age"]>=99).show()
data.filter((col("age")>=95) & (col("country")=="Hong Kong")).show()

+----+----------+--------------------+---------------+---+-------------+-----------+--------------------+--------------------+--------------------+--------------------+--------+------+------+----------+-------------+-------------+----------+--------------------+---------+----------+--------+-----------+------------------+--------------------+-----------------------------------+------------------------------------------+-----------------------+---------------------------+----------------------+------------------+----------------+-----------------+-----------------+
|rank|finalWorth|            category|     personName|age|      country|       city|              source|          industries|countryOfCitizenship|        organization|selfMade|status|gender| birthDate|        title|         date|     state|residenceStateRegion|birthYear|birthMonth|birthDay|cpi_country|cpi_change_country|         gdp_country|gross_tertiary_education_enrollment|gross_primary_education_enrollment_country|life_e

In [8]:
column_name = "age"
# Calculate quantiles
quantiles = data.approxQuantile(column_name, [0.25,0.75], 0.01)
# Calculate the interquartile range (IQR)
IQR = quantiles[1] - quantiles[0]
# Define the lower and upper bounds for outliers
lower_bound = quantiles[0] - 1.5 * IQR
upper_bound = quantiles[1] + 1.5 * IQR
# Filter the DataFrame to identify potential outliers
outliers = data.filter((col(column_name) < lower_bound) | (col(column_name) > upper_bound))

# Show the potential outliers
outliers.show() 

+----+----------+--------------------+--------------------+---+-------------+-------+-------------------+--------------------+--------------------+------------+--------+------+------+--------------+-----------+--------------+---------+-------------+-------+--------------------+---------+----------+--------+-----------+------------------+--------------------+-----------------------------------+------------------------------------------+-----------------------+---------------------------+----------------------+------------------+----------------+-----------------+
|rank|finalWorth|            category|          personName|age|      country|   city|             source|          industries|countryOfCitizenship|organization|selfMade|status|gender|     birthDate|   lastName|     firstName|    title|         date|  state|residenceStateRegion|birthYear|birthMonth|birthDay|cpi_country|cpi_change_country|         gdp_country|gross_tertiary_education_enrollment|gross_primary_education_enrollment_

In [10]:
lower_outliers = data.filter(col(column_name) < lower_bound)
upper_outliers=data.filter(col(column_name)> upper_bound)
lower_outliers.show()
upper_outliers.show()

+----+----------+--------------------+--------------------+---+-------------+-------+-------------------+--------------------+--------------------+------------+--------+------+------+--------------+-----------+--------------+---------+-------------+-------+--------------------+---------+----------+--------+-----------+------------------+--------------------+-----------------------------------+------------------------------------------+-----------------------+---------------------------+----------------------+------------------+----------------+-----------------+
|rank|finalWorth|            category|          personName|age|      country|   city|             source|          industries|countryOfCitizenship|organization|selfMade|status|gender|     birthDate|   lastName|     firstName|    title|         date|  state|residenceStateRegion|birthYear|birthMonth|birthDay|cpi_country|cpi_change_country|         gdp_country|gross_tertiary_education_enrollment|gross_primary_education_enrollment_

In [36]:
data.select(mean(column_name)).collect()[0][0]

65.14019417475728

In [70]:
data=data.withColumn("gdp_country",regexp_replace("gdp_country","[^0-9.]",""))
data=data.withColumn("gdp_country", col("gdp_country").cast("bigint")).select("gdp_country")

+--------------+
|   gdp_country|
+--------------+
| 2715518274227|
|21427700000000|
|21427700000000|
|21427700000000|
|21427700000000|
|21427700000000|
|21427700000000|
| 1258286717125|
| 2611000000000|
|21427700000000|
| 2715518274227|
|21427700000000|
| 1394116310769|
|21427700000000|
|19910000000000|
|21427700000000|
|21427700000000|
|21427700000000|
|21427700000000|
|21427700000000|
+--------------+
only showing top 20 rows



In [72]:
unique_counts = [(col_name, data.select(col(col_name)).distinct().count()) for col_name in data.columns]
# Print the results
for col_name, count in unique_counts:
    print(f"Number of unique values in column '{col_name}': {count}")

AttributeError: 'NoneType' object has no attribute 'columns'